In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [3]:
latent_dim = 100
img_size = 28 * 28
batch_size = 64
epochs = 5
lr = 0.0002

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 479kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.42MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.97MB/s]


In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model=nn.Sequential(
            nn.Linear(latent_dim,256),
            nn.LeakyReLU(0.2),
            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,img_size),
            nn.Tanh()
        )
    def forward(self, z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model=nn.Sequential(
            nn.Linear(img_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,1),
            nn.Sigmoid()
        )
    def forward(self, img):
        return self.model(img.view(img.size(0), -1))


    


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_Gen=Generator().to(device)
model_Dis=Discriminator().to(device)

# Loss & Optimizer for both

In [6]:
model_Dis

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [7]:
lr=0.01
optimizer_G=torch.optim.Adam(params=model_Gen.parameters(),lr=lr)
optimizer_D=torch.optim.Adam(params=model_Dis.parameters(),lr=lr)

loss_fn=nn.BCELoss()


In [8]:
x=0
for i,(img,_) in enumerate(train_loader):
    real_img=img.to(device)
    real_labels=torch.ones(img.size(0),1).to(device)
    fake_labels=torch.zeros(img.size(0),1).to(device)
    #Discriminator train
    #Real img loss
    optimizer_D.zero_grad()
    real_output=model_Dis(real_img)
    loss_disc_real=loss_fn(real_output, real_labels)
    #Fake img create & loss

    z=torch.randn(img.size(0),latent_dim).to(device)
    fake_img=model_Gen(z)
    #fake img loss
    fake_output=model_Dis(fake_img.detach())
    loss_disc_fake=loss_fn(fake_output,fake_labels)



    #total loss 
    total_loss=loss_disc_real + loss_disc_fake
    total_loss.backward()
    optimizer_D.step()

    



    
    #Generator train


    optimizer_G.zero_grad()
    validity=model_Dis(fake_img)
    Gen_loss=loss_fn(validity,real_labels)
    Gen_loss.backward()
    optimizer_G.step()
    
    
    
    
    if i % 200 == 0:
        print(f"[Batch {i}/{len(train_loader)}] "
              f"[D loss: {total_loss.item()}] [G loss: {Gen_loss.item()}]")
    #generate & save image
    with torch.no_grad():
        sample_z=torch.randn(16,latent_dim).to(device)
        generated=model_Gen(sample_z).cpu()
        plt.figure(figsize=(4,4))
        for j in range(16):
            plt.subplot(4, 4, j+1)
            plt.imshow(generated[j].view(28, 28).numpy(), cmap='gray')
            plt.axis('off')
        plt.savefig(f"gan_output_epoch.png")
        plt.close()

    
    


[Batch 0/938] [D loss: 1.4160501956939697] [G loss: 0.7777471542358398]
[Batch 200/938] [D loss: 59.375] [G loss: 40.625]
[Batch 400/938] [D loss: 79.6875] [G loss: 20.3125]
[Batch 600/938] [D loss: 90.625] [G loss: 9.375]
[Batch 800/938] [D loss: 98.4375] [G loss: 1.5625]


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

# ========== Hyperparameters ==========
latent_dim = 100
img_size = 28 * 28
batch_size = 64
lr = 0.0002
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== DataLoader ==========
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Range: [-1, 1]
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# ========== Generator ==========
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, img_size),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# ========== Discriminator ==========
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        return self.model(img_flat)

# ========== Initialize Models ==========
G = Generator().to(device)
D = Discriminator().to(device)
loss_fn = nn.BCELoss()
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr)

# ========== Training ==========
os.makedirs("gan_outputs", exist_ok=True)
fixed_noise = torch.randn(16, latent_dim).to(device)

for epoch in range(epochs):
    for i, (imgs, _) in enumerate(train_loader):
        batch_size_i = imgs.size(0)

        # Flatten and send to device
        real_imgs = imgs.view(batch_size_i, -1).to(device)

        # Labels with smoothing
        real_labels = torch.full((batch_size_i, 1), 0.9, device=device)
        fake_labels = torch.zeros(batch_size_i, 1).to(device)

        # ======== Train Discriminator ========
        optimizer_D.zero_grad()

        real_output = D(real_imgs)
        loss_real = loss_fn(real_output, real_labels)

        z = torch.randn(batch_size_i, latent_dim).to(device)
        fake_imgs = G(z)
        fake_output = D(fake_imgs.detach())
        loss_fake = loss_fn(fake_output, fake_labels)

        d_loss = loss_real + loss_fake
        d_loss.backward()
        optimizer_D.step()

        # ======== Train Generator ========
        optimizer_G.zero_grad()

        validity = D(fake_imgs)
        g_loss = loss_fn(validity, real_labels)  # Want D to think G's images are real
        g_loss.backward()
        optimizer_G.step()

        # ======== Logs ========
        if i % 200 == 0:
            print(f"[Epoch {epoch+1}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    # ======== Save Sample Images ========
    with torch.no_grad():
        generated = G(fixed_noise).view(-1, 1, 28, 28)
        save_image(generated, f"gan_outputs/sample_epoch_{epoch+1}.png", normalize=True)

[Epoch 1/5] [Batch 0/938] [D loss: 1.4138] [G loss: 0.6861]
[Epoch 1/5] [Batch 200/938] [D loss: 0.5197] [G loss: 1.8503]
[Epoch 1/5] [Batch 400/938] [D loss: 0.6540] [G loss: 2.5600]
[Epoch 1/5] [Batch 600/938] [D loss: 0.7472] [G loss: 2.9349]
[Epoch 1/5] [Batch 800/938] [D loss: 0.4646] [G loss: 3.2542]
[Epoch 2/5] [Batch 0/938] [D loss: 1.3427] [G loss: 1.2567]
[Epoch 2/5] [Batch 200/938] [D loss: 1.2465] [G loss: 1.5435]
[Epoch 2/5] [Batch 400/938] [D loss: 0.5029] [G loss: 3.2766]
[Epoch 2/5] [Batch 600/938] [D loss: 1.1806] [G loss: 2.2818]
[Epoch 2/5] [Batch 800/938] [D loss: 0.5852] [G loss: 2.2243]
[Epoch 3/5] [Batch 0/938] [D loss: 1.2744] [G loss: 1.4221]
[Epoch 3/5] [Batch 200/938] [D loss: 1.9331] [G loss: 0.8820]
[Epoch 3/5] [Batch 400/938] [D loss: 0.5408] [G loss: 2.2207]
[Epoch 3/5] [Batch 600/938] [D loss: 1.2022] [G loss: 1.1568]
[Epoch 3/5] [Batch 800/938] [D loss: 1.0409] [G loss: 1.8425]
[Epoch 4/5] [Batch 0/938] [D loss: 0.9141] [G loss: 1.9793]
[Epoch 4/5] [Bat